
# Laboratorio 2: Análisis de sensibilidad en problemas de Programación Lineal con Python

En este laboratorio, exploraremos el análisis de sensibilidad en problemas de Programación Lineal (PL) utilizando Python y la biblioteca Pyomo que ya introdujimos en la primera sesión de laboratorio. Como hemos aprendido, el **análisis de sensibilidad** es una herramienta que permite entender cómo los cambios en los parámetros de un modelo pueden afectar su solución óptima. 

En el contexto de la PL, este análisis se centra generalmente en dos aspectos:
1. Cambios en los **lados derechos de las restricciones** (es decir, en la disponibilidad de recursos).
2. Cambios en los **coeficientes de la función objetivo** (que representan el beneficio o costo de las variables de decisión).

Aunque Pyomo no cuenta con herramientas específicas para realizar análisis de sensibilidad directamente, podemos implementarlo modificando los parámetros del modelo y observando el efecto en la solución.

Pyomo permite resolver problemas de programación lineal, pero no tiene funcionalidades nativas específicas para realizar análisis de sensibilidad como las que podrías encontrar en paquetes más especializados en optimización, como Gurobi o CPLEX. Sin embargo, existen algunas estrategias que puedes usar para obtener información sobre el análisis de sensibilidad de un  problema de PL. 

Por otro lado, GLPK, que es el solver que introdujimos en el Laboratorio 1, es una herramienta robusta para resolver problemas de programación lineal y entera, pero, desafortunadamente, tiene limitaciones para realizar análisis de sensibilidad, ya que no proporciona directamente precios sombra ni rangos de factibilidad en su salida. Sin embargo, podemos llevar a cabo una estrategia de análisis de sensibilidad manual con GLPK en Pyomo.




## Análisis de sensibilidad sobre los lados derechos de las restricciones: cálculos de precios sombra

El análisis de sensibilidad sobre **los lados derechos de las restricciones** permite determinar cómo cambia la solución óptima cuando varían los límites de las restricciones, tales como la disponibilidad de recursos o capacidades. Esto es especialmente útil para entender la robustez de una solución ante cambios en la disponibilidad de recursos. GLPK puede proporcionar valores duales en ciertos problemas de programación lineal.

1. **Configura el modelo en Pyomo** tal como lo harías normalmente, definiendo las restricciones y la función objetivo.
2. **Activa la opción de valores duales** en Pyomo añadiendo un `Suffix` al modelo:
   ```python
   model.dual = Suffix(direction=Suffix.IMPORT)
   ```
   En Pyomo, un Suffix es una estructura que permite asociar datos adicionales a componentes de un modelo, como restricciones o variables, y se utiliza comúnmente para trabajar con información complementaria que generan los solucionadores, como los valores duales (o precios sombra) en problemas de optimización lineal.

   ¿Para qué sirve Suffix?

   1. Acceso a valores duales: En problemas de optimización lineal, los solucionadores pueden calcular valores duales para las restricciones activas. Los valores duales indican el cambio en el valor de la función objetivo si se incrementa el límite de una restricción en una unidad. Estos valores son fundamentales para el análisis de sensibilidad.

   2. Datos adicionales del solucionador: Algunos solucionadores generan datos adicionales (como los precios sombra, el estado de las restricciones, u otras métricas de diagnóstico) que se pueden capturar y almacenar usando `Suffix`.

   Sintaxis de Suffix

   Para crear un `Suffix`, se usa la siguiente sintaxis:
   ```python
   model.suffix_name = Suffix(direction=Suffix.IMPORT)
   ```

   Aquí:
   - `model.suffix_name`: es el nombre que asignas al sufijo, por ejemplo, `model.dual`.
   - `direction=Suffix.IMPORT`: indica que el sufijo se importa desde el solucionador al modelo en Pyomo. Esto significa que Pyomo llenará el sufijo con datos provenientes del solucionador. También existen otros valores posibles para `direction`, como `Suffix.EXPORT`, que permite enviar datos desde Pyomo al solucionador.

3. **Extrae los valores duales** después de resolver el modelo con GLPK. Los valores duales estarán disponibles en `model.dual[constraint]`, siempre que el problema sea lineal y GLPK pueda calcularlos.

### Ejemplo

Aquí tienes una adaptación de un ejemplo básico para un modelo de producción que utiliza GLPK y extrae los valores duales para realizar un análisis de sensibilidad:



In [1]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir variables de decisión
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

# Definir función objetivo
model.obj = Objective(expr=40*model.x + 30*model.y, sense=maximize)

# Definir restricciones
model.demand = Constraint(expr=model.x <= 40)
model.laborA = Constraint(expr=model.x + model.y <= 80)
model.laborB = Constraint(expr=2*model.x + model.y <= 100)

# Añadir el sufijo dual para obtener los valores duales
model.dual = Suffix(direction=Suffix.IMPORT)

# Resolver el modelo usando GLPK
solver = SolverFactory('glpk')
solver.solve(model)

# Imprimir la solución
print("x =", model.x())
print("y =", model.y())
print("Valor de la función objetivo =", model.obj())

# Realizar el análisis de sensibilidad con los valores duales
print("\nAnálisis de sensibilidad:")
print("Dual de la restricción de demanda:", model.dual[model.demand])
print("Dual de la restricción de laborA:", model.dual[model.laborA])
print("Dual de la restricción de laborB:", model.dual[model.laborB])


x = 20.0
y = 60.0
Valor de la función objetivo = 2600.0

Análisis de sensibilidad:
Dual de la restricción de demanda: 0.0
Dual de la restricción de laborA: 20.0
Dual de la restricción de laborB: 10.0


### Limitaciones
Es importante señalar que GLPK solo proporciona valores duales en problemas lineales y no siempre tiene la capacidad de devolver información tan completa como otros solucionadores, como Gurobi o CPLEX. Sin embargo, para análisis de sensibilidad básicos, como el cálculo de precios sombra, es suficiente.


## Análisis de sensibilidad sobre los coeficientes de la función objetivo

También es posible realizar análisis de sensibilidad sobre los **coeficientes de la función objetivo** en un problema de programación lineal usando el solver GLPK y la herramienta de formulación de problemas Pyomo. Esto implica analizar cómo varía el valor óptimo y la solución del modelo cuando se modifican los coeficientes de la función objetivo. Aunque Pyomo no ofrece un análisis de sensibilidad de estos coeficientes directamente, hay maneras de hacerlo manualmente.

### Procedimiento para realizar el análisis de sensibilidad de los coeficientes de la función objetivo

Para analizar cómo afectan los cambios en los coeficientes de la función objetivo a la solución óptima, puedes seguir estos pasos:

1. **Resolver el modelo base**: Primero, resuelve el modelo con los coeficientes originales de la función objetivo y guarda la solución óptima y el valor de la función objetivo.

2. **Modificar los coeficientes de la función objetivo**: Incrementa o disminuye cada coeficiente de la función objetivo de manera controlada usando bucles. Puedes hacerlo en un rango de valores para observar cómo afecta cada cambio a la solución óptima y al valor de la función objetivo.

3. **Re-optimizar el modelo**: Resuelve el modelo nuevamente después de cada modificación de un coeficiente. Registra el nuevo valor de la función objetivo y la nueva solución.

4. **Analizar los resultados**: Compara los resultados obtenidos en cada iteración con los de la solución base. Esto te dará una idea de la sensibilidad de la solución óptima respecto a los cambios en los coeficientes de la función objetivo.

### Ejemplo

Aquí tienes un ejemplo sencillo de cómo hacer esto con Pyomo:


In [2]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir variables de decisión
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

# Función objetivo inicial
model.obj = Objective(expr=40 * model.x + 30 * model.y, sense=maximize)

# Definir restricciones
model.demand = Constraint(expr=model.x <= 40)
model.laborA = Constraint(expr=model.x + model.y <= 80)
model.laborB = Constraint(expr=2 * model.x + model.y <= 100)

# Resolver el modelo base
solver = SolverFactory('glpk')
solver.solve(model)

# Guardar los valores iniciales
original_obj_value = model.obj()
original_x = model.x()
original_y = model.y()

print("Solución base:")
print("Valor objetivo:", original_obj_value)
print("x =", original_x)
print("y =", original_y)

# Análisis de sensibilidad sobre el coeficiente de x en la función objetivo
coef_range = range(30, 51, 5)  # Cambiar el coeficiente de 30 a 50 en pasos de 5
sens_results = []

for coef_x in coef_range:
    # Actualizar el coeficiente de x en la función objetivo
    model.obj.set_value(coef_x * model.x + 30 * model.y)
    
    # Resolver el modelo modificado
    solver.solve(model)
    
    # Guardar los resultados
    sens_results.append({
        'coef_x': coef_x,
        'obj_value': model.obj(),
        'x': model.x(),
        'y': model.y()
    })

# Mostrar resultados de sensibilidad
print("\nAnálisis de sensibilidad en el coeficiente de x en la función objetivo:")
for result in sens_results:
    print(f"Coeficiente de x: {result['coef_x']}, Valor objetivo: {result['obj_value']}, x = {result['x']}, y = {result['y']}")

Solución base:
Valor objetivo: 2600.0
x = 20.0
y = 60.0

Análisis de sensibilidad en el coeficiente de x en la función objetivo:
Coeficiente de x: 30, Valor objetivo: 2400.0, x = 20.0, y = 60.0
Coeficiente de x: 35, Valor objetivo: 2500.0, x = 20.0, y = 60.0
Coeficiente de x: 40, Valor objetivo: 2600.0, x = 20.0, y = 60.0
Coeficiente de x: 45, Valor objetivo: 2700.0, x = 20.0, y = 60.0
Coeficiente de x: 50, Valor objetivo: 2800.0, x = 20.0, y = 60.0


### Limitaciones

Este análisis manual tiene algunas limitaciones:
- Es computacionalmente costoso si tienes muchos coeficientes o si deseas evaluar múltiples valores de cambio.
- No proporciona rangos exactos (como el análisis de rango) para los coeficientes en los que la solución óptima se mantiene constante.

Para obtener un análisis de rango exacto de los coeficientes de la función objetivo, algunos solucionadores avanzados (como CPLEX y Gurobi) ofrecen esta funcionalidad de manera automática. Sin embargo, en Pyomo, un análisis de sensibilidad manual es una buena aproximación y puede ser suficiente para entender cómo afectan los cambios en los coeficientes a la solución del problema.

## Ejercicios de análisis de sensibilidad en problemas de PL con Pyomo.

### Ejercicio 1.

Una empresa fabrica tres productos diferentes: producto A, producto B y producto C. Para su fabricación, la empresa emplea cuatro recursos limitados: horas de trabajo de las máquinas, materia prima, capacidad de almacenamiento y horas de trabajo de los empleados. La disponibilidad mensual de cada recurso se presenta a continuación:

- Horas de trabajo de las máquinas: 600 horas.
- Materia prima: 500 kg.
- Capacidad de almacenamiento: 400 unidades.
- Horas de trabajo de empleados: 300 horas.

La siguiente tabla muestra la cantidad de recursos que se necesitan para producir una unidad de cada producto:

| Producto | Horas de máquinas | Materia prima | Almacenamiento | Horas de empleados |
|----------|--------------------|---------------|----------------|---------------------|
| A        | 8                 | 4             | 2              | 3                   |
| B        | 5                 | 6             | 4              | 2                   |
| C        | 7                 | 5             | 3              | 5                   |

El beneficio por cada unidad fabricada del producto A es de 40 €, del producto B es de 50 €, y del producto C es de 60 €. La empresa desea maximizar el beneficio total de su producción, dadas las limitaciones de recursos disponibles.

Se pide:

1. Formule el problema de producción óptima como un problema de Programación Lineal y resuélvalo.
2. Calcule los precios sombra sin resolver el problema dual e interprete los precios sombra obtenidos.
3. Formule el problema dual asociado al problema de producción óptima, resuélvalo y compare con las soluciones obtenidas en el apartado anterior.
4. Trate de determinar cuánto puede variar el beneficio unitario del producto A sin que varíe la solución óptima del problema primal. ¿Y el beneficio unitario del producto B?





#### **Apartado 1. Formulación del problema y resolución.**

In [3]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir variables de decisión
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)
model.z = Var(within=NonNegativeReals)


# Función objetivo inicial
model.obj = Objective(expr=40 * model.x + 50 * model.y + 60*model.z, sense=maximize)


# Definir restricciones
model.machines = Constraint(expr=8 * model.x + 5 * model.y + 7 * model.z <= 600)    # Horas de máquinas
model.material = Constraint(expr=4 * model.x + 6 * model.y + 5 * model.z <= 500)     # Materia prima
model.storage = Constraint(expr=2 * model.x + 4 * model.y + 3 * model.z <= 400)      # Capacidad de almacenamiento
model.labor = Constraint(expr=3 * model.x + 2 * model.y + 5 * model.z <= 300)        # Horas de empleados

# Agregar sufijo dual para almacenar precios sombra
model.dual = Suffix(direction=Suffix.IMPORT)

# Resolver el modelo base
solver = SolverFactory('glpk')
solver.solve(model)

# Guardar los valores iniciales
original_obj_value = model.obj()
original_x = model.x()
original_y = model.y()
original_z = model.z()


print("Solución base:")
print("Valor objetivo:", original_obj_value)
print("x =", original_x)
print("y =", original_y)
print("z =", original_z)

Solución base:
Valor objetivo: 4900.0
x = 0.0
y = 50.0
z = 40.0


La solución óptima del problema de producción es:
- Cantidad de producto A: $x_A = 0$ unidades.
- Cantidad de producto B: $x_B = 50$ unidades.
- Cantidad de producto C: $x_C = 40$ unidades.

El beneficio máximo que puede obtener la empresa es de $4900$ €.

#### **Apartado 2. Cálculo e interpretación de los precios sombra.**

In [4]:
print("\nAnálisis de sensibilidad:")
print("Precio sombra de la restricción de máquinas:", model.dual[model.machines])
print("Precio sombra de la restricción de materiales:", model.dual[model.material])
print("Precio sombra de la restricción de almacenamiento:", model.dual[model.storage])
print("Precio sombra de la restricción de horas de empleados:", model.dual[model.labor])


Análisis de sensibilidad:
Precio sombra de la restricción de máquinas: 0.0
Precio sombra de la restricción de materiales: 6.5
Precio sombra de la restricción de almacenamiento: 0.0
Precio sombra de la restricción de horas de empleados: 5.5


1. **Precio sombra de las máquinas:** Un precio sombra de 0 indica que este recurso es una restricción **no activa** en la solución óptima. Es decir, aumentar la disponibilidad de horas de máquinas no aumentaría el beneficio total, dado que no es un recurso limitante en este contexto.
2. **Precio sombra de la restricción de materiales:** Un precio sombra de 6.5 sugiere que, si se incrementa la disponibilidad de materia prima en una unidad adicional (por ejemplo, 1 kg), el beneficio total aumentaría en 6.5 unidades. Esta restricción estaria **activa**. Aumentar su disponibilidad podría permitir producir más de los productos que generan beneficios.
3. **Precio sombra de la restricción de almacenamiento:** Al igual que las horas de máquinas, un precio sombra de 0 indica que el almacenamiento no es un recurso limitante en la solución óptima. Es decir, es una restricción **no activa**.
4. **Precio sombra de la restricción de horas de empleados:** Un precio sombra de 5.5 indica que aumentar en una unidad las horas de trabajo de los empleados incrementaría el beneficio total en 5.5 unidades monetarias. Esta restricción estaria **activa**.

#### **Apartado 3. Formulación del problema dual y comparación con los resultados.**
$$
\text{minimizar:} \quad  600y_A + 500y_B + 400y_C + 300y_D
$$

$$
\begin{align*}
    &\text{sujeto a:} \quad 8y_A + 4y_B + 2y_C + 3y_D \geq 40, \\
    &\quad 5y_A + 6y_B + 4y_C + 2y_D \geq 50, \\
    &\quad 7y_A + 5y_B + 3y_C + 5y_D \geq 60, \\
    &\quad y_A, y_B, y_C, y_D \geq 0.
\end{align*}
$$


In [5]:
# Crear el modelo dual
dual_model = ConcreteModel()

# Variables del dual (precios sombra de cada restricción del primal)
dual_model.y1 = Var(within=NonNegativeReals)  # Horas de máquinas
dual_model.y2 = Var(within=NonNegativeReals)  # Materia prima
dual_model.y3 = Var(within=NonNegativeReals)  # Capacidad de almacenamiento
dual_model.y4 = Var(within=NonNegativeReals)  # Horas de empleados

# Función objetivo del dual: Maximizar el valor de los recursos limitantes
dual_model.obj = Objective(expr=600 * dual_model.y1 + 500 * dual_model.y2 + 400 * dual_model.y3 + 300 * dual_model.y4, sense=minimize)

# Restricciones del dual
dual_model.constraintA = Constraint(expr=8 * dual_model.y1 + 4 * dual_model.y2 + 2 * dual_model.y3 + 3 * dual_model.y4 >= 40)
dual_model.constraintB = Constraint(expr=5 * dual_model.y1 + 6 * dual_model.y2 + 4 * dual_model.y3 + 2 * dual_model.y4 >= 50)
dual_model.constraintC = Constraint(expr=7 * dual_model.y1 + 5 * dual_model.y2 + 3 * dual_model.y3 + 5 * dual_model.y4 >= 60)

# Resolver el modelo dual
solver.solve(dual_model)

# Mostrar solución del dual
print("Valor del dual:", dual_model.obj())
print("Precio sombra (y1):", dual_model.y1())
print("Precio sombra (y2):", dual_model.y2())
print("Precio sombra (y3):", dual_model.y3())
print("Precio sombra (y4):", dual_model.y4())

Valor del dual: 4900.0
Precio sombra (y1): 0.0
Precio sombra (y2): 6.5
Precio sombra (y3): 0.0
Precio sombra (y4): 5.5


#### **Apartado 4. Análisis de la sensibilidad para los beneficios unitarios.**
#### 4.1. Análisis de sensibilidad sobre el producto A.

In [6]:
# Análisis de sensibilidad sobre el coeficiente de x en la función objetivo
coef_range = range(30, 51, 5)
sens_results = []

for coef_x in coef_range:
    # Actualizar el coeficiente de x en la función objetivo
    model.obj.set_value(coef_x * model.x + 50 * model.y + 60 * model.z)
    
    # Resolver el modelo modificado
    solver.solve(model)
    
    # Guardar los resultados
    sens_results.append({
        'coef_x': coef_x,
        'obj_value': model.obj(),
        'x': model.x(),
        'y': model.y(),
        'z': model.z()
    })

# Mostrar resultados de sensibilidad
print("\nAnálisis de sensibilidad en el coeficiente de x en la función objetivo:")
for result in sens_results:
    print(f"Coeficiente de x: {result['coef_x']}, Valor objetivo: {result['obj_value']}, x = {result['x']}, y = {result['y']}, z = {result['z']}")


Análisis de sensibilidad en el coeficiente de x en la función objetivo:
Coeficiente de x: 30, Valor objetivo: 4900.0, x = 0.0, y = 50.0, z = 40.0
Coeficiente de x: 35, Valor objetivo: 4900.0, x = 0.0, y = 50.0, z = 40.0
Coeficiente de x: 40, Valor objetivo: 4900.0, x = 0.0, y = 50.0, z = 40.0
Coeficiente de x: 45, Valor objetivo: 4953.84615384615, x = 21.5384615384615, y = 44.6153846153846, z = 29.2307692307692
Coeficiente de x: 50, Valor objetivo: 5061.538461538457, x = 21.5384615384615, y = 44.6153846153846, z = 29.2307692307692


### Ejercicio 2.

Un médico ha recetado a una paciente una pauta alimenticia que se basa en tres alimentos: arroz, pescado y verduras frescas. La combinación de estos alimentos debe cumplir con unos requisitos mínimos de nutrientes, concretamente en términos de proteínas y calorías. Los requisitos mínimos son:

- Proteínas: al menos 3 unidades.
- Calorías: al menos 4,000 calorías.

Cada alimento aporta una cantidad específica de proteínas y calorías por kilogramo, como se muestra en la siguiente tabla:

| Alimento          | Proteínas (unidades/kg) | Calorías (kcal/kg) | Coste (unidades monetarias/kg) |
|-------------------|-------------------------|---------------------|--------------------------------|
| Arroz             | 1                       | 2,000              | 55                             |
| Pescado           | 3                       | 3,000              | 125                            |
| Verduras frescas  | 2                       | 1,000              | 55                             |

La paciente desea diseñar una dieta que minimice el coste total, respetando los requisitos de nutrientes impuestos por el médico.

Se pide:

1. Formule el problema de optimización de la dieta como un problema de Programación Lineal que minimice el coste de los alimentos mientras se cumplen los requisitos de nutrientes.

2. Resuelva el problema de Programación Lineal para determinar la cantidad óptima de cada alimento en la dieta.

3. Analice cómo afecta al coste total un aumento del 10% en el coste del arroz. ¿Se modifica la solución óptima del problema primal con este cambio?

4. Formule el problema dual asociado a la optimización de la dieta y resuélvalo. Interprete los precios sombra obtenidos y explique su significado en el contexto de este problema. ¿Qué implicaciones tiene un precio sombra positivo en los requisitos de nutrientes?


In [7]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir variables de decisión
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)
model.z = Var(within=NonNegativeReals)


# Función objetivo inicial
model.obj = Objective(expr=55 * model.x + 125 * model.y + 55*model.z, sense=minimize)

# Definir restricciones
model.demand = Constraint(expr= model.x + 3 * model.y + 2*model.z >= 3)
model.laborA = Constraint(expr= 2000 * model.x + 3000 * model.y + 1000 * model.z >= 4000)

# Resolver el modelo base
solver = SolverFactory('glpk')
solver.solve(model)

# Guardar los valores iniciales
original_obj_value = model.obj()
original_x = model.x()
original_y = model.y()
original_z = model.z()


print("Solución base:")
print("Valor objetivo:", original_obj_value)
print("x =", original_x)
print("y =", original_y)
print("z =", original_z)


# Análisis de sensibilidad sobre el coeficiente de x en la función objetivo
coef_range = range(50, 70, 5)
sens_results = []

for coef_x in coef_range:
    # Actualizar el coeficiente de x en la función objetivo
    model.obj.set_value(coef_x * model.x + 125 * model.y + 55 * model.z)
    
    # Resolver el modelo modificado
    solver.solve(model)
    
    # Guardar los resultados
    sens_results.append({
        'coef_x': coef_x,
        'obj_value': model.obj(),
        'x': model.x(),
        'y': model.y(),
        'z': model.z()
    })

# Mostrar resultados de sensibilidad
print("\nAnálisis de sensibilidad en el coeficiente de x en la función objetivo:")
for result in sens_results:
    print(f"Coeficiente de x: {result['coef_x']}, Valor objetivo: {result['obj_value']}, x = {result['x']}, y = {result['y']}, z = {result['z']}")

Solución base:
Valor objetivo: 128.33333333333354
x = 1.66666666666667
y = 0.0
z = 0.666666666666667

Análisis de sensibilidad en el coeficiente de x en la función objetivo:
Coeficiente de x: 50, Valor objetivo: 120.00000000000018, x = 1.66666666666667, y = 0.0, z = 0.666666666666667
Coeficiente de x: 55, Valor objetivo: 128.33333333333354, x = 1.66666666666667, y = 0.0, z = 0.666666666666667
Coeficiente de x: 60, Valor objetivo: 136.66666666666688, x = 1.66666666666667, y = 0.0, z = 0.666666666666667
Coeficiente de x: 65, Valor objetivo: 145.00000000000023, x = 1.66666666666667, y = 0.0, z = 0.666666666666667
